# This notebook defines a class named Plots, that contains the code for generating the plots in the main notebook Power Supply vs Microbial Survival
## The code in this notebook calls functions contained in a notebook named Utils (included in the present repository)

# Importing libraries and modules 

In [ ]:
import numpy as np
import random 
import matplotlib.pyplot as plt 
import Utils as U # Containins customized functions for plot generation 

# Defining the class

In [ ]:
class Plots():
    def __init__(self):
        
        #Model constants:
        
        self.rmax = 1.7*(10**(-16)) # mol s^(-1)
        self.P0 = 10**(-13) # J s^(-1)
        self.k = 10**(-6) # mol cm^(-3)
        self.Ps = 10**(-10) # J s^(-1) cm^(-3)
        self.Lamb = 10**(-8) # s^(-1)
        self.R = 8.31 # J mol^(-1) K^(-1)
        self.T = 300 # K
        
        #Model variables:
        
        self.E = np.arange(10**3,10**5+1,10**3) # Energy scale (measured in J)
        self.N = np.arange(10,1000+1,50) # Number of consumers
        self.pH = np.arange(0.,14.01,0.01) # pH
    
    ## H_B vs H_P plots:
    
    def HBvsHP(self,sigma=np.array([1]),
               subst_flat=True,gibbs_flat=True,
               tradeoff=False):
        """
        This function generates the graphics for H_B (Survival diversity) 
        vs H_P (Power Supply diversity) 
        
        Args:
        ----------
        Sigma : np.array of floats, optional
            Widths for the model parameter distributions. 
            The default is np.array([1]).
        
        subst_flat : boolean, optional
            Whether the distribution for input substrate 
            concentrations is flat or not. The default is True.
        
        gibbs_flat : boolean, optional
            Whether the distribution for Gibbs energies of 
            reactions is flat or not. The default is True.
        
        tradeoff : boolean, optional
            Whether an efficiency-cost tradeoff is considered or not. 
            The default is False.

        Returns
        -------
        None. 
        
        """
        
        # Constatns
        
        rmax = self.rmax
        P0 = self.P0
        k = self.k 
        R = self.R
        T = self.T 
        
        # Variables

        E = self.E
        N = self.N  
        
        # Auxiliary variables
        S0 = np.zeros(len(N))
        Substrates = [[[[0] for j in range(len(sigma))] 
                       for iE in range(len(E)) ] 
                      for iN in range(len(N))]

        # Outcomes

        bio_ent = np.zeros([len(N),len(E),len(sigma)])
        power_ent = np.zeros([len(N),len(E),len(sigma)])
         

        # computing steady state substrate concentrations

        for j in range(len(sigma)):
            s = sigma[j]
            for iE in range(len(E)):
                E0 = E[iE]
                for iN in range(len(N)):
                    n = N[iN]
                    x = np.arange(1,n+1,1,dtype = int)
             
                    r,p = U.rp_profiles((x, n, s, 
                                 tradeoff), 
                                 P0 = P0,rmax = rmax)
                    E0i = U.gibbs_profile((x, s, n, 
                               gibbs_flat),E0 = E0)
             
                    arg1 = k*p/(r*R*T)
                    arg2 = (r*E0i-p)/(r*R*T)
                    seqi = arg1/U.LW(arg1,arg2)
                    Substrates[iN][iE][j] = seqi  
                    S0[iN] = np.max([S0[iN],np.max(seqi)])
             
        # Computing survival diversity and power supply diversity
        
        for j in range(len(sigma)):
            s = sigma[j]
            for iE in range(len(E)):
                E0 = E[iE]
                for iN in range(len(N)):
                    n = N[iN]
                    x = np.arange(1,n+1,1,dtype = int)
             
                    r,p = U.rp_profiles((x, n, s, 
                                 tradeoff), 
                                 P0 = P0,rmax = rmax)
                                
                    E0i = U.gibbs_profile((x, s, n, 
                                 gibbs_flat),E0 = E0)
                                
                    phi = U.subst_profile(x,n,s,
                                subst_flat,S0 = S0[iN])
             
                    #
                    Eeq = E0i + R*T*(np.log(Substrates[iN][iE][j]))
                    b = Eeq*(phi-np.exp((Eeq-E0i)/(R*T)))/p
                    b = b[b>0]
                    bio_ent[iN,iE,j] = U.H(b/sum(b))
                    power_ent[iN,iE,j] = U.H(phi*Eeq/sum(phi*Eeq))  
             
             
         ## Generating the plots

        U.HBvsHP(bio_ent, power_ent, sigma, 
                 subst_flat, gibbs_flat, tradeoff)

        return None 
    
    
    
    ####################################
    ###################################


    # HB vs HP along a pH gradient
    
    def pH_HBvsHP(self,sigma=np.array([1]),
               subst_flat=True,gibbs_flat=True,
               tradeoff=False):
        """
        This function generates the graphics for H_B vs H_P 
        
        Parameters
        ----------
        Sigma : np.array of floats, optional
            Widths for the model parameter distributions. 
            The default is np.array([1]).
        
        subst_flat : boolean, optional
            Whether the distribution for input substrate 
            concentrations is flat or not. The default is True.
        
        gibbs_flat : boolean, optional
            Whether the distribution for Gibbs energies of 
            reactions is flat or not. The default is True.
        
        tradeoff : boolean, optional
            Whether an efficiency-cost tradeoff is considred or not. 
            The default is False.

        Returns
        -------
        None. 
        

        """
        
        # Constatns
        
        rmax = self.rmax
        P0 = self.P0
        k = self.k 
        R = self.R
        T = self.T 
        
        # Variables

        pH = self.pH
        E = np.arange(10**5,(10**6)+10**3,10**3) # Energy scale (measured in J)
        n = 100 # Number of habitats

        Ratio = 3
        x = np.arange(1,n+1)
        stoich = np.random.randint(1,11, size = n)
        signs = np.ones(n)
        neg_ind = random.sample(range(n),int(0.5*n)) 
        signs[neg_ind] = - 1

        # Outcomes

        bio_ent = np.zeros([len(pH),len(E),len(sigma)])
        power_ent = np.zeros([len(pH),len(E),len(sigma)])

        # Computing survival diversity

        for j in range(len(sigma)):
            s = sigma[j]
            phihat = 1.2*np.ones(n)
            rhat = np.ones(n)
            phat = np.ones(n)
            e0i = np.ones(n)
    
            if not(subst_flat):
                phihat = 10**3*np.exp(-(x-n/2)**2/(s**2*n**2))+1.2
    
            if not(gibbs_flat):
                e0i = np.exp(-(x-n/2)**2/(5*s**2*n**2)) + 1/6
                Ratio = 1
    
            if tradeoff:
                rhat = np.exp(-(x-n/2)**2/(s**2*n**2/2))+1/100
                phat = np.exp(-(x-n/2)**2/(s**2*n**2/2))+1/20
    
            r = rmax*rhat
            p = P0*phat

            S0 = 0
            for iE in range(len(E)):
                E0 = E[iE]
                arg1 = k*p/(r*R*T)
                arg2 = (E0*e0i*r-p)/(r*R*T)
                S0 = max(S0,max(arg1/U.LW(arg1,arg2)))
    
            for iE in range(len(E)):
                E0 = E[iE]
                dG0producers = (E0*e0i/Ratio)*(signs>0)
                dG0consumers = (E0*e0i)*(signs<0)
                dG0 = dG0producers + dG0consumers
        
                for ipH in range(len(pH)):         
                    ph = pH[ipH]
                    dG = dG0 + signs*stoich*R*T*np.log(10)*ph
                    dG[dG<0] = 0
                    
                    arg1 = k*p/(r*R*T)
                    arg2 = (r*dG-p)/(r*R*T)
                    seqi = arg1/U.LW(arg1,arg2)
                    dG = dG + R*T*np.log(seqi)
            
                    b = (S0*phihat-seqi)*dG/p
                    b = b[b>0]
            
                    bio_ent[ipH,iE,j] = U.H(b/sum(b))
                    power_ent[ipH,iE,j] = U.H(dG/sum(dG))
            
        ## Generating the plots

        U.HBvsHP(bio_ent, power_ent, sigma, 
                 subst_flat, gibbs_flat, tradeoff,
                 PH=True)
        
        return None
    
    
    
    ####################################
    ###################################

    
    
    
    ## Computation of survival probability  
        
    def steady_state(self,sigma=np.array([1]),
               subst_flat=True,gibbs_flat=True,
               tradeoff=False):
        
        """
        Computes the survival probability as a function of the energy scale
        and for several distributions of model parameters
        
        
        Args:
        ----------
        Sigma : np.array of floats, optional
            Widths for the model parameter distributions. 
            The default is np.array([1]).
        
        subst_flat : boolean, optional
            Whether the distribution for input substrate 
            concentrations is flat or not. The default is True.
        
        gibbs_flat : boolean, optional
            Whether the distribution for Gibbs energies of 
            reactions is flat or not. The default is True.
        
        tradeoff : boolean, optional
            Whether an efficiency-cost tradeoff is considred or not. 
            The default is False.

        Returns
        -------
        surv_prob : np.array containing the survival probability
        
        Steady_state_E : np.array containing the steady state available energy
        
        max_index : np.array containing the index of the most viable habitat
        
        sigma : float. The width of the distribution of model parameters
        
        """
        
        # Constants
        
        rmax = self.rmax
        P0 = self.P0
        k = self.k 
        R = self.R
        T = self.T 

        # Variables

        E = self.E
        n = 50 # Number of habitats 
        
        # Boolean variable 
        
        combined = bool(tradeoff*(not(subst_flat))*(not(gibbs_flat)))
        
        # Auxiliary variables
        S0 = np.zeros([len(sigma)]) 
        Substrates = [[[0] for j in range(len(sigma))] 
                      for iE in range(len(E))]

        # Outcomes

        surv_prob = np.zeros([len(E),len(sigma),n])
        Steady_state_E = np.zeros([len(E),len(sigma),n]) # steady state available energy 
        max_index = np.zeros((len(E),len(sigma))) # index of the most abundant species
         

        # Computing steady state substrate concentrations 

        for j in range(len(sigma)):
            s = sigma[j]
            for iE in range(len(E)):
                E0 = E[iE]
                x = np.arange(1,n+1,1,dtype = int)
         
                r,p = U.rp_profiles((x, n, s, 
                           tradeoff, 
                           combined),P0=P0,rmax=rmax)
                
                E0i = U.gibbs_profile((x, s, n, 
                         gibbs_flat, 
                         combined),E0=E0)
         
                arg1 = k*p/(r*R*T)
                arg2 = (r*E0i-p)/(r*R*T)
                seqi = arg1/U.LW(arg1,arg2)
                Substrates[iE][j] = seqi  
                S0[j] = np.max([S0[j],np.max(seqi)])
             
        # Computing survival probabilities, steady state available energy
        # and index of most viable habitat 
        
        for j in range(len(sigma)):
            s = sigma[j]
            for iE in range(len(E)):
                E0 = E[iE]
                x = np.arange(1,n+1,1,dtype = int)
         
                r,p = U.rp_profiles((x, n, s, 
                           tradeoff, 
                           combined),P0=P0,rmax=rmax)
                
                E0i = U.gibbs_profile((x, s, n, 
                         gibbs_flat, 
                         combined),E0=E0)
                
                phi = U.subst_profile(x, n, s, 
                           subst_flat,S0 = S0[j])
                #
                Eeq = E0i + R*T*(np.log(Substrates[iE][j]))
                b = Eeq*(phi-np.exp((Eeq-E0i)/(R*T)))/p
                b = b[b>0]
                surv_prob[iE,j,:] = b/sum(b)
                Steady_state_E[iE,j,:] = Eeq/E0
                max_index[iE,j] = np.argmax(surv_prob[iE,j,0:25])+1  
             
        return surv_prob, Steady_state_E, max_index, sigma
    
    
    
    
   
    ####################################
    ###################################
    
    
    
    
    ## Survival Probability plots

    
    def survival_prob_vs_energy_plots(self,data,
                                     avail_energy=False,
                                     most_viable=False):
        
        """
        Generates the plots for survival probability vs energy
        
        
        Args:
        ----------
        data : tuple
            Outcome of the function steady_sate()
        
        avail_energy : boolean, optional
            True if the generated plot shows the available
            energy at steady state. The default is False.
        
        most_viable : boolean, optional
            True if the generated plot shows the most viable habitat. 
            The default is False.

        Returns
        -------
        None. 
        
        """
        E = 10**(-3)*self.E 
        
        LS = 15 # label size
        
        surv_prob, Steady_state_E, max_index, sigma = data 
        
        if avail_energy:
            if len(sigma)==1:
                for l in range(25):
                    plt.plot(E,Steady_state_E[:,0,l],color=[(24-l)/24,0,l/24])
                plt.title('steady-state avail. energy, $s$='+'{:.3f}'.format(sigma[0]))
                plt.xlabel('$-\Delta G^0_r\,\mathrm{(kJ/mol)}$',fontsize=LS)
                plt.ylabel('$-E_{eq}/\Delta G^0_r$',fontsize=LS)   
                plt.show()
            else:
                fig,ax = plt.subplots(2,2)
                for i in range(min(len(sigma),4)):
                    for l in range(25):
                        ax[int(i/2),i-2*int(i/2)].plot(E,Steady_state_E[:,i,l],color=[(24-l)/24,0,l/24])
                    ax[int(i/2),i-2*int(i/2)].set_title('$s$='+'{:.3f}'.format(sigma[i]))
                    ax[int(i/2),i-2*int(i/2)].set_xlabel('$-\Delta G^0_r\,\mathrm{(kJ/mol)}$',fontsize=LS)
                    ax[int(i/2),i-2*int(i/2)].set_ylabel('$-E_{eq}/\Delta G^0_r$',fontsize=LS)
                for a in ax.flat:
                    a.label_outer() 
                plt.show() 
        else:
            if most_viable:
                if len(sigma)==1:
                    plt.plot(E,max_index[:,0])
                    plt.title('most viable habitat, $s$='+'{:.3f}'.format(sigma[0]))
                    plt.xlabel('$-\Delta G^0_r\,\mathrm{(kJ/mol)}$',fontsize=LS)
                    plt.ylabel('$i_{max}$',fontsize=LS)
                    plt.show() 
                else:
                    fig,ax = plt.subplots(2,2)
                    for i in range(min(len(sigma),4)):
                        ax[int(i/2),i-2*int(i/2)].plot(E,max_index[:,i])
                        ax[int(i/2),i-2*int(i/2)].set_title('$s$='+'{:.3f}'.format(sigma[i]))
                        ax[int(i/2),i-2*int(i/2)].set_xlabel('$-\Delta G^0_r\,\mathrm{(kJ/mol)}$',fontsize=LS)
                        ax[int(i/2),i-2*int(i/2)].set_ylabel('$i_{max}$',fontsize=LS)
                    for a in ax.flat:
                        a.label_outer() 
                    plt.show() 
                   
            else:
                if len(sigma)==1:
                    for l in range(25):
                        plt.plot(E,surv_prob[:,0,l],color=[(24-l)/24,0,l/24])
                    plt.title('survival probability, $s$='+'{:.3f}'.format(sigma[0]))
                    plt.xlabel('$-\Delta G^0_r\,\mathrm{(kJ/mol)}$',fontsize=LS)
                    plt.ylabel('$c_i^*/\sum_j c_j^*$',fontsize=LS)
                    plt.show() 
                else:
                    fig,ax = plt.subplots(2,2)
                    for i in range(min(len(sigma),4)):
                        for l in range(25):
                            ax[int(i/2),i-2*int(i/2)].plot(E,surv_prob[:,i,l],color=[(24-l)/24,0,l/24])
                        ax[int(i/2),i-2*int(i/2)].set_title('$s$='+'{:.3f}'.format(sigma[i]))
                        ax[int(i/2),i-2*int(i/2)].set_xlabel('$-\Delta G^0_r\,\mathrm{(kJ/mol)}$',fontsize=LS)
                        ax[int(i/2),i-2*int(i/2)].set_ylabel('$c_i^*/\sum_j c_j^*$',fontsize=LS)
                    for a in ax.flat:
                        a.label_outer() 
                    plt.show()
        return None 



  ######################################
  #####################################
  
  
  
  # Survival Probability plots (dependence on pH)
  
    def pH_survival_prob_vs_energy_plots(self,sigma=0.1,Ratio=3,
               subst_flat=True,gibbs_flat=True,
               tradeoff=False):
        
        """
        Generates the plots for survival probability vs energy scale 
        when the dependence on pH is considered 
        
        
        Args:
        ----------
        S : float, optional
            Width for the model parameter distributions. 
            The default is np.array([1]).
        
        Ratio : float, optional
             Ratio available energy H-consumers to
                   available energy H-producers 
        
        subst_flat : boolean, optional
            Whether the distribution for input substrate 
            concentrations is flat or not. The default is True.
        
        gibbs_flat : boolean, optional
            Whether the distribution for Gibbs energies of 
            reactions is flat or not. The default is True.
        
        tradeoff : boolean, optional
            Whether an efficiency-cost tradeoff is considred or not. 
            The default is False.

        
        Returns
        ------- 
        None
        """
        
        # Constatns
        
        rmax = self.rmax
        P0 = self.P0
        k = self.k 
        R = self.R
        T = self.T 
        Lamb = self.Lamb 
        
        # Variables

        pH = self.pH # pH
        E0 = 10**6 # J/mol 
        n = 100 # number of secies 
        arg1 = k*P0/(rmax*R*T)
        arg2 = (rmax*E0-P0)/(rmax*R*T)
        S0 = 5*arg1/U.LW(arg1,arg2)
        x = np.arange(1,n+1)
        stoich = np.random.randint(1,11, size = n)

        phi = np.ones(n)
        r = np.ones(n)
        p = np.ones(n)
        E0i = np.ones(n)
        
        # Outcomes 
        bio_ent = np.zeros(len(pH))
        power_ent = np.zeros(len(pH))
        abundances = np.zeros((len(pH),n))
        Eeq = np.zeros((len(pH),n))
        
        combined = bool(tradeoff*(not(subst_flat))*(not(gibbs_flat)))
        if combined:
            phi = 10**3*np.exp(-(x-n/2)**2/(sigma**2*n**2))+1.2
            r = np.exp(-(x-n/2)**2/(sigma**2*n**2/2))+1/100
            p = np.exp(-(x-n/2)**2/(sigma**2*n**2/2))+1/20
            E0i = np.exp(-(x-n/2)**2/(5*sigma**2*n**2)) + 1/6
            Ratio = 1
        else:
            if not(subst_flat):
                phi = 10**3*np.exp(-(x-n/2)**2/(sigma**2*n**2))+1.2
            if not(gibbs_flat):
                E0i = np.exp(-(x-n/2)**2/(5*sigma**2*n**2)) + 1/6
            if tradeoff:
                r = np.exp(-(x-n/2)**2/(sigma**2*n**2/2))+1/100
                p = np.exp(-(x-n/2)**2/(sigma**2*n**2/2))+1/20
       

        r = rmax*r
        p = P0*p
        E0i = E0*E0i

        signs = np.ones(n)
        neg_ind = random.sample(range(n),int(0.5*n)) 
        signs[neg_ind] = - 1
        dG0producers = (E0i/Ratio)*(signs>0) 
        dG0consumers = E0i*(signs<0)
        dG0 = dG0producers + dG0consumers

        for j in range(len(pH)):
            ph = pH[j]
            dG = dG0 + signs*stoich*R*T*np.log(10)*ph
            dG[dG<0] = 0 
           
            arg1 = k*p/(r*R*T)
            arg2 = (r*dG-p)/(r*R*T)
            seqi = arg1/U.LW(arg1,arg2)
            dG = dG + R*T*np.log(seqi)
            Eeq[j,:] = dG
           
            b = (S0*phi-seqi)*(Lamb*dG/p);
            b[b<0] = 0
            abundances[j,:] = b 
           
            bio_ent[j] = U.H(b/sum(b))
            power_ent[j] = U.H(dG/sum(dG))
    

        ## Generating the plots

        data = (pH,abundances,bio_ent,power_ent,Eeq/E0,signs)
        U.pH_plots(data)

    
        return None

